# Modeling

In [1]:
from operator import mod
from os import getcwd
from os.path import exists, join

import joblib
from sklearn.datasets import fetch_california_housing
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.svm import SVR
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport
import random
import ast


In [2]:
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

from sklearn.linear_model import LogisticRegression, LinearRegression
import warnings
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
from sklearn.ensemble import  GradientBoostingClassifier

import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC, LinearSVC 
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import recall_score

from sklearn import tree
from sklearn.decomposition import PCA, SparsePCA

from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
import json
import pickle
from IPython.display import Image
import warnings
from sklearn.metrics import classification_report
from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import RobustScaler, KBinsDiscretizer



In [3]:
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from collections import Counter

from tqdm import tqdm

tqdm.pandas()

In [4]:
# Depression screener
dep_screener_cols = [
    'little_interest_in_doing_things',
    'feeling_down_depressed_hopeless',
    'trouble_falling_or_staying_asleep',
    'feeling_tired_or_having_little_energy',
    'poor_appetitie_or_overeating',
    'feeling_bad_about_yourself',
    'trouble_concentrating',
    'moving_or_speaking_to_slowly_or_fast',
    'thoughts_you_would_be_better_off_dead',
    'difficult_doing_daytoday_tasks'
]
model_features_opt2 = dep_screener_cols + [
    'seen_mental_health_professional',
    'times_with_12plus_alc',
    'time_since_last_healthcare',
    'cholesterol_prescription',
    'high_cholesterol',
    'age_in_years',
    'horomones_not_bc',
    'months_since_birth',
    'arthritis',
    'high_bp',
    'regular_periods',
    'moderate_recreation',
    'thyroid_issues',
    'vigorous_recreation',
    'stroke',
    'is_usa_born',
    'asthma',
    'count_days_moderate_recreational_activity',
    'have_health_insurance',
    'num_dep_screener_0',
    'weight_lbs_over_height_in_ratio'
]

model_features_low_opt7 = [
    'count_days_seen_doctor_12mo_bin',
    'times_with_12plus_alc',
    'seen_mental_health_professional',
    'count_lost_10plus_pounds',
    'arthritis',
    'horomones_not_bc',
    'is_usa_born',
    'times_with_8plus_alc',
    'time_since_last_healthcare',
    'duration_last_healthcare_visit',
    'work_schedule'
]

columns_to_use_low = model_features_low_opt7
columns_to_use_high = model_features_opt2

# Opt 9: Ensemble Model

Build 2 models with different feature set
- Model 1: 
 - GB trained on observations with 1+ dep screener response. 
 - Uses features from opt 2. 
 - Uses undersampler.
- Model 2: 
 - RF trained on observations with 0 dep screener response. 
 - Uses features from opt 7. 
 - Uses undersampler

Notes
- _low = has 9+ dep screeners answered 0
- _high = has <9 dep screeners answered 0

In [6]:
# v2 csv
df_cdc_clean = pd.read_csv('../../data/cdc_nhanes_survey_responses_clean.csv')

# filter to pregnant moms
cdc_survey_pmom = df_cdc_clean[df_cdc_clean['has_been_pregnant'] == 1]
print(cdc_survey_pmom.shape)
## ##################################################################
## Dep screener cnt

cdc_survey_pmom['num_dep_screener_0'] = (cdc_survey_pmom[dep_screener_cols]==0).sum(axis=1)
cdc_survey_pmom['weight_lbs_over_height_in_ratio'] = round(cdc_survey_pmom['weight_lbs'] / cdc_survey_pmom['height_in'],1)

# add bins

feature_values = cdc_survey_pmom['count_days_seen_doctor_12mo'].dropna().values
feature_values = feature_values.reshape([feature_values.shape[0],1])

# create bins using estimator
est = KBinsDiscretizer(
    n_bins=10,
    encode='ordinal', 
    strategy='uniform', 
    subsample=None
)
est.fit(feature_values)
feature_values = est.transform(feature_values)

fill_arr = cdc_survey_pmom['count_days_seen_doctor_12mo'].values.copy()
fill_arr[~np.isnan(fill_arr)] = np.asarray([val[0] for val in feature_values])
cdc_survey_pmom['count_days_seen_doctor_12mo_bin'] = fill_arr





# subset to features and do preprocessing
imputer = SimpleImputer(strategy='median')  
trans = RobustScaler()


data_low_dep_screener = cdc_survey_pmom[cdc_survey_pmom['num_dep_screener_0'] >= 9].copy()
data_low_dep_screener = data_low_dep_screener[['MDD'] + columns_to_use_low]

y_low = data_low_dep_screener['MDD'].values
x_low = data_low_dep_screener.iloc[:,1:].values
x_low = imputer.fit_transform(x_low)
x_low = trans.fit_transform(x_low)
x_train_low, x_test_low, y_train_low, y_test_low = train_test_split(
    x_low, 
    y_low, 
    test_size=0.2, 
    random_state=42
) 

data_high_dep_screener = cdc_survey_pmom[cdc_survey_pmom['num_dep_screener_0'] < 9].copy()
data_high_dep_screener = data_high_dep_screener[['MDD'] + columns_to_use_high]
y_high = data_high_dep_screener['MDD'].values
x_high = data_high_dep_screener.iloc[:,1:].values
x_high = imputer.fit_transform(x_high)
x_high = trans.fit_transform(x_high)
x_train_high, x_test_high, y_train_high, y_test_high = train_test_split(
    x_high, 
    y_high, 
    test_size=0.2, 
    random_state=42
) 



/var/folders/0k/kvbyl6wn7kl2qbt7_94pckjw0000gn/T/ipykernel_42108/3063162359.py:2: DtypeWarning: Columns (664,665) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cdc_clean = pd.read_csv('../../data/cdc_nhanes_survey_responses_clean.csv')
/var/folders/0k/kvbyl6wn7kl2qbt7_94pckjw0000gn/T/ipykernel_42108/3063162359.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdc_survey_pmom['num_dep_screener_0'] = (cdc_survey_pmom[dep_screener_cols]==0).sum(axis=1)
/var/folders/0k/kvbyl6wn7kl2qbt7_94pckjw0000gn/T/ipykernel_42108/3063162359.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

(7741, 863)


In [45]:
class CustomBin(TransformerMixin):
    def __init__(self, some_stuff=None, column_names= []):
        pass
        # self.some_stuff = some_stuff
        self.column_names = column_names
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # do stuff on X, and return dataframe
        # of the same shape - this gets messy
        # if the preceding item is a numpy array
        # and not a dataframe
        if isinstance(X, np.ndarray):
            X = pd.DataFrame(X, columns=self.column_names)
        feature_values = X['count_days_seen_doctor_12mo'].dropna().values
        feature_values = feature_values.reshape([feature_values.shape[0],1])

        # create bins using estimator
        est = KBinsDiscretizer(
            n_bins=10,
            encode='ordinal', 
            strategy='uniform', 
            subsample=None
        )
        est.fit(feature_values)
        feature_values = est.transform(feature_values)

        fill_arr = X['count_days_seen_doctor_12mo'].values.copy()
        fill_arr[~np.isnan(fill_arr)] = np.asarray([val[0] for val in feature_values])
        X['count_days_seen_doctor_12mo_bin'] = fill_arr

        return X


# # using this by itself works as well
# my_pipeline = make_pipeline(CustomBin(column_names=["my_str", "val"]))
# my_pipeline.fit_transform(cdc_survey_pmom)


In [46]:

class CustomFeatures(TransformerMixin):
    def __init__(self, some_stuff=None, column_names= []):
        pass
        # self.some_stuff = some_stuff
        self.column_names = column_names
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # do stuff on X, and return dataframe
        # of the same shape - this gets messy
        # if the preceding item is a numpy array
        # and not a dataframe
        if isinstance(X, np.ndarray):
            X = pd.DataFrame(X, columns=self.column_names)
        X['num_dep_screener_0'] = (X[dep_screener_cols]==0).sum(axis=1)
        X['weight_lbs_over_height_in_ratio'] = round(X['weight_lbs'] / X['height_in'],1)

        return X


# # using this by itself works as well
# my_pipeline = make_pipeline(CustomFeatures(column_names=["my_str", "val"]))
# my_pipeline.fit_transform(cdc_survey_pmom)

In [7]:
## #####################################################
## Model 1: Based on observations with 1+ dep screener non-zero response

# partially correct for class imbalance
rus_model1 = RandomUnderSampler(
    random_state=42, 
    sampling_strategy=1,
    replacement=False
)
x_train_high_rus, y_train_high_rus = rus_model1.fit_resample(x_train_high,y_train_high)
print(f"x_train_high_rus: {x_train_high_rus.shape}")

gb = GradientBoostingClassifier(random_state=42)
gb.fit(x_train_high_rus, y_train_high_rus)
y_pred_high = gb.predict(x_test_high)

gb = GradientBoostingClassifier(random_state=42)
gb.fit(x_train_high_rus, y_train_high_rus)
y_pred_high = gb.predict(x_test_high)

## #####################################################
## Model 1: Based on observations with 0 dep screener non-zero response

# partially correct for class imbalance
rus = RandomUnderSampler(
    random_state=42, 
    sampling_strategy=0.12,
    replacement=False
)
x_train_low_rus, y_train_low_rus = rus.fit_resample(x_train_low,y_train_low)
print(f"x_train_low_rus: {x_train_low_rus.shape}")

rf = RandomForestClassifier(random_state=42)
rf.fit(x_train_low_rus, y_train_low_rus)
y_pred_low = rf.predict(x_test_low)

## #####################################################
## Predict: 

y_pred = np.concatenate((y_pred_high, y_pred_low))
y_actual = np.concatenate((y_test_high, y_test_low))

pd.DataFrame(classification_report(y_actual,y_pred,output_dict=True))


x_train_high_rus: (982, 31)
x_train_low_rus: (849, 11)


,0,1,accuracy,macro avg,weighted avg
precision,0.956664,0.358896,0.830859,0.657780,0.891060
recall,0.848441,0.688235,0.830859,0.768338,0.830859
f1-score,0.899308,0.471774,0.830859,0.685541,0.852387
support,1379.000000,170.000000,0.830859,1549.000000,1549.000000


In [61]:

data_low_dep_screener = cdc_survey_pmom[cdc_survey_pmom['num_dep_screener_0'] >= 9].copy()
data_low_dep_screener = data_low_dep_screener[['MDD'] + columns_to_use_low]

y_low = data_low_dep_screener['MDD'].values
x_low = data_low_dep_screener.iloc[:,1:].values
x_low = imputer.fit_transform(x_low)
x_low = trans.fit_transform(x_low)
x_train_low, x_test_low, y_train_low, y_test_low = train_test_split(
    x_low, 
    y_low, 
    test_size=0.2, 
    random_state=42
) 

rus_model1 = RandomUnderSampler(
    random_state=42, 
    sampling_strategy=1,
    replacement=False
)
x_train_high_rus, y_train_high_rus = rus_model1.fit_resample(x_train_high,y_train_high)
print(f"x_train_high_rus: {x_train_high_rus.shape}")

gb = GradientBoostingClassifier(random_state=42)
gb.fit(x_train_high_rus, y_train_high_rus)
y_pred_high = gb.predict(x_test_high)


# processing_pipeline = make_pipeline(
#     SimpleImputer(strategy='most_frequent'),
#     RobustScaler(), 
#     GradientBoostingClassifier(random_state=42)
# )

processing_pipeline = make_pipeline(
    CustomFeatures(CustomBin(column_names=['MDD'] + columns_to_use_low)),
    CustomBin(CustomBin(column_names=['MDD'] + columns_to_use_low)),
    SimpleImputer(strategy='median') , 
    RobustScaler(),
    # RandomUnderSampler(
    #     random_state=42, 
    #     sampling_strategy=0.12,
    #     replacement=False
    # ), 
    RandomForestClassifier(random_state=42)
)



processing_pipeline.fit(x_train_high_rus)
# pred_labels  = processing_pipeline.predict(x_test)
# pred_labels = [x.round() for x in pred_labels]


ValueError: This RandomForestClassifier estimator requires y to be passed, but the target y is None.

In [ ]:
processing_pipeline = make_pipeline(SimpleImputer(strategy='most_frequent'), RobustScaler(), GradientBoostingClassifier(random_state=42))

model_filename = "model_pipeline.pkl"
model_path = join(getcwd(), model_filename)
processing_pipeline.fit(x_train, y_train)
pred_labels  = processing_pipeline.predict(x_test)
pred_labels = [x.round() for x in pred_labels]
df_gnb = pd.DataFrame(classification_report(
    y_test, 
    pred_labels, 
    # target_names=target_names, 
    output_dict=True
))

In [27]:

x_train, x_test, y_train, y_test = get_model_data(
    original_df = df_cdc_clean,
    columns = cols_to_keep
)

processing_pipeline = make_pipeline(SimpleImputer(strategy='most_frequent'), RobustScaler(), GaussianNB())

model_filename = "model_pipeline.pkl"
model_path = join(getcwd(), model_filename)
if not exists(model_path):
    processing_pipeline.fit(x_train, y_train)
    pred_labels  = processing_pipeline.predict(x_test)
    pred_labels = [x.round() for x in pred_labels]
    df_gnb = pd.DataFrame(classification_report(
                y_test, 
                pred_labels, 
                # target_names=target_names, 
                output_dict=True
    ))
    joblib.dump(processing_pipeline, model_path)
    print('successfully trained model')
else:
    print("Model has already been trained, no need to rerun")

df_gnb

NameError: name 'cols_to_keep' is not defined

In [ ]:
try:
    !rm trainer.py
except:
    pass

rm: trainer.py: No such file or directory


In [ ]:
try:
    !jupyter nbconvert --no-prompt --to script trainer.ipynb
except:
    pass

[NbConvertApp] Converting notebook trainer.ipynb to script
[NbConvertApp] Writing 26351 bytes to trainer.py
